<a href="https://colab.research.google.com/github/santteegt/om-fol-timeseries/blob/master/CNN_Model_Chest_device.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from tqdm.notebook import tqdm

%matplotlib inline

In [2]:
!pip install torch-lr-finder

In [3]:
# !wget https://github.com/santteegt/om-fol-timeseries/archive/master.zip -O om-fol-timeseries.zip && unzip om-fol-timeseries.zip && rm om-fol-timeseries.zip
# !wget https://github.com/santteegt/om-fol-timeseries/raw/master/segmented_data/WESAD_segmented.zip && unzip -d data WESAD_segmented.zip && rm WESAD_segmented.zip
# from google.colab import files
# uploaded = files.upload()

!gdown --id 1Esb60gAz0E75eVheHjcUYgk495k79HZJ  # segmented dataset on my Google drive

Downloading...
From: https://drive.google.com/uc?id=1Esb60gAz0E75eVheHjcUYgk495k79HZJ
To: /content/all_raw.csv
22.2MB [00:00, 135MB/s]


In [4]:
 !ls -l

total 21652
-rw-r--r-- 1 root root 22166015 Jul  6 13:45 all_raw.csv
drwxr-xr-x 1 root root     4096 Jun 26 16:26 sample_data


## Load Subject Data

In [5]:
# BASE_PATH = './data'
# [subject for subject in os.listdir(BASE_PATH) if subject.endswith('.feather')]

In [6]:
df = pd.read_csv('all_raw.csv')
df.shape

(132433, 10)

In [7]:
df.head()

,ACC_x,ACC_y,ACC_z,ECG,EDA,EMG,RESP,TEMP,label,subject
0,0.804085,0.129889,-0.428761,-0.003546,1.694510,0.000016,2.529596,34.957394,3,S8
1,0.804379,0.135955,-0.424341,-0.001006,1.698947,-0.000009,2.993713,34.954926,3,S8
2,0.808023,0.148010,-0.412429,0.017687,1.716327,-0.000002,2.968826,34.954052,3,S8
3,0.813003,0.155346,-0.401195,0.001699,1.740343,0.000009,2.363336,34.954731,3,S8
4,0.819421,0.158451,-0.379938,-0.001678,1.785164,0.000005,1.212493,34.955471,3,S8


## Data Loader

In [8]:
class WESADDataset(Dataset):
    #Constructor is mandatory
    def __init__(self, dataframe, transform=None):
        # normalizer = StandardScaler()
        normalizer = MinMaxScaler()
        self.dataframe = dataframe.drop(columns=['subject','label'])
        self.X = normalizer.fit_transform(self.dataframe.astype(np.float32))
        self.labels = dataframe['label']
        self.transform = transform # e.g. torch.Tensor
    
    def to_torchtensor(self):            
        self.dataframe = torch.from_numpy(self.dataframe)
        self.labels = torch.from_numpy(self.labels)
    
    def __len__(self):
        #Mandatory
        '''Returns:
                Length [int]: Length of Dataset/batches
        '''
        return self.dataframe.shape[0]

    def __getitem__(self, idx): 
        #Mandatory 
        
        '''Returns:
                    Data [Torch Tensor]: 
                    Target [ Torch Tensor]:
        '''
        # sample = self.dataframe.iloc[idx].astype(np.float32).to_numpy()
        sample = self.X[idx]
        target = self.labels[idx]
                
        if self.transform:
            sample = self.transform(sample)

        return sample, target

In [9]:
def get_data_loader(subject, train_batch_size=128, val_batch_size=5):

    train = WESADDataset(df[df['subject'] != subject].reset_index(drop=True))
    test = WESADDataset(df[df['subject'] == subject].reset_index(drop=True))

    train_dl = torch.utils.data.DataLoader(train, batch_size=train_batch_size, shuffle=True, pin_memory=True, num_workers=4)
    val_dl = torch.utils.data.DataLoader(test, batch_size=val_batch_size, shuffle=False, pin_memory=True, num_workers=4)
    
    return train_dl, val_dl

In [10]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)


class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

## Methods & utils for model training

In [11]:
from sklearn.metrics import f1_score

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

def f1(output, label, threshold=0.5, beta=1):
    probs = torch.argmax(output.data.to('cpu'), dim=1)
    preds = label.data.to('cpu')
    return f1_score(preds, probs, average='macro')

def plot_scores(history):
    accuracies = [x['val_acc'] for x in history]
    f1 = [x['val_f1'] for x in history]
    plt.plot(accuracies, '-x', label='Accuracy')
    plt.plot(f1, '-x', label='F1')
    plt.xlabel('epoch')
    plt.ylabel('Score')
    plt.title('Score vs. No. of epochs')
    plt.legend();

def plot_losses(history):
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs');

In [12]:
class BaseModel(nn.Module):
    def training_step(self, batch):
        data, labels = batch 
        out = self(data)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        data, labels = batch 
        out = self(data)                    # Generate predictions
        # print('out', out.shape)
        loss = F.cross_entropy(out, labels)   # Calculate loss
        # print('loss', loss.shape)
        out = F.softmax(out, dim=1)           # Apply Softmax
        print('val_softmax', out.shape)
        acc = accuracy(out, labels)           # Calculate accuracy
        f1_ = f1(out, labels)                  # Calculate F1
        return {'val_loss': loss.detach(), 'val_acc': acc, 'val_f1': f1_}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        batch_f1 = [x['val_f1'] for x in outputs]
        # print('batch_f1', len(batch_f1))
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        epoch_f1 = np.mean(np.array(batch_f1))         # Combine F1 scores
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item(), 'val_f1': epoch_f1.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, train_acc: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}, val_f1: {:.4f}".format(
            epoch, result['train_loss'], result['train_acc'], result['val_loss'], result['val_acc'], result['val_f1']))

In [13]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, optimizer):
    history = []
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_accs = []
        train_losses = []
        for batch in tqdm(train_loader):
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            data, labels = batch
            train_out = F.softmax(model(data), dim=1)
            train_accs.append(accuracy(train_out, labels))


        # Validation phase
        # if epoch % 10 == 0:
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['train_acc'] = torch.stack(train_accs).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history

## Models

In [14]:
class WesadCNN(BaseModel):
    def __init__(self, input_dim, hidden_dim, output_dim=3, lstm_layers=1):
        super().__init__()
        self.network1 = nn.Sequential(
            nn.Conv1d(in_channels=8, out_channels=32, kernel_size=1, stride=1, padding=0),
            nn.ReLU(),
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=1, stride=1, padding=0),
            nn.ReLU(),

        )
        self.lstm = nn.LSTM(input_size=64, hidden_size=64, num_layers=lstm_layers, dropout=0.)
        self.conv = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=1, stride=1, padding=0)
        self.classifier = nn.Sequential(
            nn.Linear(128, 64),
            nn.Linear(64, 32),
            nn.Linear(32, 3)
        )

    def forward(self, x):
        xb = x.view(-1, 8, 1)
        out = self.network1(xb)
        print('out', out.shape)
        out, _ = self.lstm(out.view(-1, 1, 64))
        out = F.dropout(out, p=0.5)
        print('out', out.shape)
        out = self.conv(out.permute(0, 2, 1))
        print('out', out.shape)
        out = F.relu(out)
        flatten = out.view(out.size(0), -1)
        print('flatten', flatten.shape)
        out = self.classifier(flatten)
        print('out', out.shape)
        return out


## General Model Parameters

In [15]:
subjects = df['subject'].unique()
subjects.sort()
device = get_default_device()

train_batch_size = 128
val_batch_size = 256
input_dim = df.drop(columns=['subject', 'label']).shape[1]
output_dim = 3
lstm_layers = 5

In [16]:
# lrs = [0.5, 0.1, 0.01, 0.001]
# epochs = [20, 20, 20,  20]

In [17]:
# from torch_lr_finder import LRFinder

# max_lr = 1e-5
# # grad_clip = 0.001
# # weight_decay = 1e-4
# opt_func = torch.optim.Adam
# criterion = F.cross_entropy
# train_dl, val_dl = get_data_loader(subject='S2', train_batch_size=train_batch_size, val_batch_size=val_batch_size)

# model = WesadCNN(input_dim=input_dim, hidden_dim=input_dim, output_dim=output_dim, lstm_layers=lstm_layers)
# # optimizer = opt_func(model.parameters(), lr=max_lr, weight_decay=weight_decay)
# optimizer = opt_func(model.parameters(), lr=max_lr)
# lr_finder = LRFinder(model, optimizer, criterion, device="cuda")
# lr_finder.range_test(train_dl, end_lr=10000, num_iter=1000)
# lr_finder.plot() # to inspect the loss-learning rate graph
# # lr_finder.reset() # to reset the model and optimizer to their initial state

## Running models - LOSO CV

In [18]:
models = []
for subject in subjects:
    print('LOSO', subject)
    train_dl, val_dl = get_data_loader(subject, train_batch_size=train_batch_size, val_batch_size=val_batch_size)
    train_ddl = DeviceDataLoader(train_dl, device)
    val_ddl = DeviceDataLoader(val_dl, device)
    model = to_device(WesadCNN(input_dim=input_dim, hidden_dim=input_dim, output_dim=output_dim, lstm_layers=1), device)
    break

LOSO S10


In [20]:
def try_batch(dl):
    for data, labels in dl:
        # print(data)
        # print(labels)
        print('images.shape:', data.shape)
        out = model(data)
        print('out.shape:', out.shape)
        print('out[0]:', out[0])
        print('softmax(out[0]):', F.softmax(out, dim=1)[0])
        _, p = torch.max(F.softmax(out, dim=1), dim=1)
        print(p[0])
        break

try_batch(train_ddl)

images.shape: torch.Size([128, 8])
out torch.Size([128, 64, 1])
out torch.Size([128, 1, 64])
out torch.Size([128, 128, 1])
flatten torch.Size([128, 128])
out torch.Size([128, 3])
out.shape: torch.Size([128, 3])
out[0]: tensor([ 0.0619, -0.0653, -0.0291], device='cuda:0', grad_fn=<SelectBackward>)
softmax(out[0]): tensor([0.3580, 0.3152, 0.3268], device='cuda:0', grad_fn=<SelectBackward>)
tensor(0, device='cuda:0')


In [20]:
history = [evaluate(model, val_ddl)]
history

out torch.Size([256, 64, 1])
out torch.Size([256, 1, 64])
out torch.Size([256, 128, 1])
flatten torch.Size([256, 128])
out torch.Size([256, 3])
val_softmax torch.Size([256, 3])


RuntimeError: ignored

In [ ]:
epochs = 5
lr = 5e-3
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
history += fit(epochs, lr, model, train_ddl, val_ddl, optimizer)


Epoch [0], train_loss: 1.0532, val_loss: 1.1090, val_acc: 0.4102



Epoch [1], train_loss: 0.9966, val_loss: 0.9997, val_acc: 0.5508



Epoch [2], train_loss: 0.9959, val_loss: 0.9997, val_acc: 0.5508



Epoch [3], train_loss: 0.9960, val_loss: 0.9997, val_acc: 0.5508



Epoch [4], train_loss: 0.9960, val_loss: 0.9997, val_acc: 0.5508


In [ ]:
plot_scores(history[1:])

torch.float32

In [ ]:
plot_losses(history[1:])